In [3]:
# ==============================================================================
# SCRIPT FINAL DE ENGENHARIA DE FEATURES
# ==============================================================================

import pandas as pd
from sqlalchemy import create_engine
from web3 import Web3
import json
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Passo 1: Carregar os Dados Brutos do Banco de Dados ---
print("--- Passo 1: Carregando dados da Mainnet do seu servidor ---")
SERVER_IP = "34.79.99.180" 
db_connection_str = f'postgresql://admin:supersecret@{SERVER_IP}:5433/mempool_data'
df = pd.DataFrame()
try:
    db_engine = create_engine(db_connection_str)
    # Selecionamos apenas o hash e o inputData para começar
    df = pd.read_sql('SELECT hash, "inputData" FROM transactions', con=db_engine)
    # Pegamos uma amostra para o processo não ser muito longo
    df_sample = df.sample(n=min(1000, len(df)), random_state=42).copy()
    print(f"Sucesso! Carregamos uma amostra de {len(df_sample)} transações para enriquecer e analisar.")
except Exception as e:
    print(f"--- ERRO ao carregar dados do banco: {e} ---")

# --- Passo 2: Enriquecer e Decodificar Cada Transação ---
if not df_sample.empty:
    print("\n--- Passo 2: Buscando recibos e decodificando dados... (Isso pode levar vários minutos) ---")
    
    alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/OPbbmuEc74NysSD6jK28Z"
    w3 = Web3(Web3.HTTPProvider(alchemy_url))

    # ABIs para decodificar
    router_abi = json.loads('[{"name":"swapExactTokensForTokens","inputs":[{"type":"uint256","name":"amountIn"},{"type":"address[]","name":"path"}],"type":"function"}]')
    pair_abi = json.loads('[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount0Out","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1Out","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Swap","type":"event"}]')
    
    generic_router_contract = w3.eth.contract(abi=router_abi)
    
    # Assinatura do evento de Swap da Uniswap V2 (keccak-256 hash)
    SWAP_EVENT_SIGNATURE = "0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822"

    def process_transaction(row):
        tx_hash = row['hash']
        input_data = row['inputData']
        
        try:
            # Decodifica o Input Data
            input_bytes = bytes.fromhex(input_data[2:])
            func_obj, func_params = generic_router_contract.decode_function_input(input_bytes)
            
            # Se não for um swap, retorna nulo
            if 'swap' not in func_obj.fn_name.lower():
                return None
            
            # Pega o recibo da transação
            receipt = w3.eth.get_transaction_receipt(tx_hash)
            
            # Procura o evento de Swap nos logs
            amount_out = 0
            for log_entry in receipt['logs']:
                if log_entry['topics'][0].hex() == SWAP_EVENT_SIGNATURE:
                    # O evento Swap na V2 tem os valores de entrada e saída nos dados
                    amounts = w3.eth.codec.decode(['uint256', 'uint256', 'uint256', 'uint256'], log_entry['data'])
                    amount0Out, amount1Out = amounts[2], amounts[3]
                    # O amountOut real é o que não for zero
                    amount_out = amount0Out if amount0Out > 0 else amount1Out
                    break
            
            # Extrai as features
            path = func_params.get('path', [])
            token_in = path[0] if path else None
            token_out = path[-1] if path else None
            amount_in = func_params.get('amountIn', 0)
            
            return {
                'hash': tx_hash,
                'function_name': func_obj.fn_name,
                'token_in': token_in,
                'token_out': token_out,
                'amount_in': amount_in,
                'amount_out_real': amount_out
            }

        except Exception:
            return None

    # Processamento concorrente para acelerar
    tasks = []
    processed_results = []
    with ThreadPoolExecutor(max_workers=50) as executor:
        for index, row in df_sample.iterrows():
            tasks.append(executor.submit(process_transaction, row))
        for future in tqdm(as_completed(tasks), total=len(tasks)):
            result = future.result()
            if result: # Adiciona apenas os resultados que não são nulos
                processed_results.append(result)

    # Cria o DataFrame final com as features
    features_df = pd.DataFrame(processed_results)
    
    print("\n--- Dataset Final com Features de Nível Profissional ---")
    if not features_df.empty:
        # Feature final: O preço efetivo do swap
        # Adicionamos 1 para evitar divisão por zero
        features_df['effective_price'] = features_df['amount_out_real'] / (features_df['amount_in'] + 1)
        display(features_df.head())
    else:
        print("Nenhum swap simples (swapExactTokensForTokens) foi encontrado na amostra analisada.")

else:
    print("Nenhum dado carregado do banco.")

--- Passo 1: Carregando dados da Mainnet do seu servidor ---
Sucesso! Carregamos uma amostra de 1000 transações para enriquecer e analisar.

--- Passo 2: Buscando recibos e decodificando dados... (Isso pode levar vários minutos) ---


  0%|          | 0/1000 [00:00<?, ?it/s]


--- Dataset Final com Features de Nível Profissional ---
Nenhum swap simples (swapExactTokensForTokens) foi encontrado na amostra analisada.
